# Quickstart Guide

Following the Quickstart Guide. I don't have a personal OpenAI account so I will try to use HuggingfaceHub instead

https://python.langchain.com/en/latest/getting_started/getting_started.html

In [1]:
# load .env
import os
from dotenv import load_dotenv

load_dotenv()

# check HF token and SERPAPI key have been loaded
assert "OPENAI_API_KEY" in os.environ
assert "SERPAPI_API_KEY" in os.environ

## Building a Language Model Application: LLMs

### LLMs: Get predictions from a language model

In [2]:
from langchain.llms import OpenAI

In [4]:
llm = OpenAI(temperature=0.9)

In [5]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



BoldSox.


### Promp Templates: Manage prompts for LLMs

In [6]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [7]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


### Chains: Combine LLMs and prompts in multi-step workflows

In [8]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

chain.run("colorful socks")

'\n\nSocktastic Creations.'

In [9]:
chain.run("witty mugs")

'\n\nMugWit'

### Agents: Dynamically Call Chains Based on User Input

In [10]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("When did Real Madrid win La Decima? What is that number multiplied by 3?")



> Entering new AgentExecutor chain...
 I need to find out when Real Madrid won La Decima and then do a calculation
Action: Search
Action Input: "Real Madrid La Decima"
Observation: The 2014 UEFA Champions League final was the final match of the 2013–14 UEFA Champions League, the 59th season of Europe's premier club football tournament organised by UEFA, and the 22nd season since it was renamed from the European Champion Clubs' Cup to the UEFA Champions League.
Thought: I need to do a calculation
Action: Calculator
Action Input: 10 x 3
Observation: Answer: 30

Thought: I now know the final answer
Final Answer: Real Madrid won La Decima in 2014 and that number multiplied by 3 is 30.

> Finished chain.


'Real Madrid won La Decima in 2014 and that number multiplied by 3 is 30.'

At least it works now! It seems it got the inputs to the calculator wrong though

### Memory: Add State to Chains and Agents

In [11]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [12]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

This is pretty cool :)

## Building a Language Model Application: Chat Models

### Get message completions from a Chat Model

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

In [19]:
print(chat([HumanMessage(content="Translate this sentence from English to Spanish. I love programming.")]).content)

Amo programar.


In [18]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Spanish. You always provide a few alternatives for each query."),
    HumanMessage(content="Translate this sentence from English to Spanish. I love programming.")
]
print(chat(messages).content)

Me encanta programar. 

Other alternatives: 
- Amo programar.
- Me gusta mucho programar.


### Chat Prompt Templates

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}. You always provide a few alternatives for each query."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
print(chat(chat_prompt.format_prompt(input_language="English", output_language="Spanish", text="I love programming.").to_messages()).content)

Me encanta programar.
Me encanta la programación.
Amo programar.
